#  DATA REVIEW

## Imports Libraries

In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
import cufflinks as cf
%matplotlib inline 

from scipy import stats


pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.width', 1000)

import warnings
warnings.filterwarnings("ignore")

## Import Local `utils1` Modul

In [2]:
#  FONKSIYONLARIMIZI DERLEDIGIMIZ `UTILS` ADINDAKI MODULU BU NOTEBOOK ICINE IMPORT EDIYORUZ

import sys

# utils1.py dosyasının bulunduğu dizini ham dize olarak ekleyin
sys.path.append("C:/Users/duygu/Desktop/00-GitHub-Repo/00-1INTERNSHIP/INT1_PROJECT/UTILITIES/")

# 'utils1.py' dosyasında olusturulan Utils sınıfını import et
from utils1 import Utils, UtilsAnalysis

# utils1 icindeki tum class'lardan bir nesne oluştur
utils = Utils()
utils_analysis = UtilsAnalysis()

In [3]:
# utils nesnesindeki tüm fonksiyonları ve öznitelikleri listelemek için
dir(utils)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'df',
 'read_excel_page',
 'save_and_load_csv']

In [4]:
# utils nesnesindeki tüm fonksiyonları ve öznitelikleri listelemek için
dir(UtilsAnalysis)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'detect_anomalies',
 'duplicate_values',
 'find_non_numeric_values',
 'get_value_count',
 'missing_values',
 'na_ratio_plot',
 'numeric_summary',
 'object_summary',
 'rename_columns_by_position',
 'set_df']

## User Defined Funcs for Summary of Data

In [42]:
# =============== User-Defined-Function ==========================

def rename_columns_by_position(df, new_column_names):
    """
    DataFrame'deki sütun adlarını pozisyona göre yeniden adlandırır.

    Parameters:
        df (pd.DataFrame): Sütun adlarını değiştirmek istediğiniz DataFrame.
        new_column_names (list): Yeni sütun adlarının listesi.

    Returns:
        pd.DataFrame: Yeni sütun adlarıyla güncellenmiş DataFrame.
    """
    # Sütun sayısının uyumlu olup olmadığını kontrol et
    if len(new_column_names) != len(df.columns):
        raise ValueError("Sütun sayısı ile yeni adların sayısı eşleşmiyor.")
    
    # Yeni sütun adlarını atama
    df.columns = new_column_names   
    return df


#================ Categorical Features Summary ===================
def object_summary(df):
    obs = df.shape[0]
    duplicate_count = df.duplicated().sum()

    # Kategorik sütunlar için özetleme
    object_df = df.select_dtypes(include='object')
    
    # Yeni bir boş DataFrame oluşturma
    summary_df = pd.DataFrame(index=object_df.columns)

    summary_df['Dtype'] = object_df.dtypes
    summary_df['Counts'] = object_df.count()
    summary_df['Nulls'] = object_df.isnull().sum()
    summary_df['NullPercent'] = (object_df.isnull().sum() / obs) * 100
    summary_df['Top'] = object_df.apply(lambda x: x.mode().iloc[0] if not x.mode().empty else '-')
    summary_df['Frequency'] = object_df.apply(lambda x: x.value_counts().max() if not x.value_counts().empty else '-')
    summary_df['Uniques'] = object_df.nunique()

    # UniqueValues sütununu kontrol ederek ekleme (dize olarak)
    summary_df['UniqueValues'] = object_df.apply(
        lambda x: ', '.join(map(str, x.unique()[:10])) + '...' if x.nunique() > 10 else ', '.join(map(str, x.unique()))
    )

    # DataFrame şekli ve tekrar eden satır sayısını ekrana yazdırma
    print(f'1. Data shape (rows, columns): {df.shape}')
    print(f'2. Number of duplicate rows: {duplicate_count}')
    return summary_df


#================ Numerical Features Summary ===================
def numeric_summary(df):
    obs = df.shape[0]
    duplicate_count = df.duplicated().sum()

    # Numerik sütunlar için özetleme
    numeric_df = df.select_dtypes(include=['float64', 'int64'])

    # Yeni bir boş DataFrame oluşturma
    summary_df = pd.DataFrame(index=numeric_df.columns)

    summary_df['Dtype'] = numeric_df.dtypes
    summary_df['Counts'] = numeric_df.count()
    summary_df['Nulls'] = numeric_df.isnull().sum()
    summary_df['NullPercent'] = (numeric_df.isnull().sum() / obs) * 100
    summary_df['Mean'] = numeric_df.mean()
    summary_df['Std'] = numeric_df.std()
    summary_df['Min'] = numeric_df.min()
    summary_df['25%'] = numeric_df.quantile(0.25)
    summary_df['50% (Median)'] = numeric_df.median()
    summary_df['75%'] = numeric_df.quantile(0.75)
    summary_df['Max'] = numeric_df.max()

    # DataFrame şekli ve tekrar eden satır sayısını ekrana yazdırma
    print(f'1. Data shape (rows, columns): {df.shape}')
    print(f'2. Number of duplicate rows: {duplicate_count}')
    return summary_df
    

#========== Get count and percentage of values for each column =================
def get_value_count(df, column_name):
    """
    This function calculates and returns a DataFrame with the value counts and 
    their corresponding percentages for a specified column in the DataFrame.
    """
    
    vc = df[column_name].value_counts()
    vc_norm = df[column_name].value_counts(normalize=True)
    
    vc = vc.rename_axis(column_name).reset_index(name='counts')
    vc_norm = vc_norm.rename_axis(column_name).reset_index(name='percent')
    vc_norm['percent'] = (vc_norm['percent'] * 100).map('{:.2f}%'.format)
    
    df_result = pd.concat([vc[column_name], vc['counts'], vc_norm['percent']], axis=1)
    return df_result


#============== Checks duplicates and drops them ==========================

def duplicate_values(df):
    print("Duplicate check...")
    num_duplicates = df.duplicated(subset=None, keep='first').sum()
    if num_duplicates > 0:
        print("There are", num_duplicates, "duplicated observations in the dataset.")
        df.drop_duplicates(keep='first', inplace=True)
        print(num_duplicates, "duplicates were dropped!")
        print("No more duplicate rows!")
    else:
        print("There are no duplicated observations in the dataset.")



# ========== User-Defined-Function for Missing Values ============
def missing_values(df):
    """This function calculates the missing values count and their percentage in a DataFrame."""

    missing_count = df.isnull().sum()
    value_count = df.isnull().count()
    missing_percentage = round(missing_count / value_count * 100, 2)
    
    # Format the percentage as '0.00%' with % symbol
    missing_percentage_formatted = missing_percentage.map("{:.2f}%".format)
    # Create a DataFrame to store the results
    missing_df = pd.DataFrame({"count": missing_count, "percentage": missing_percentage_formatted}) 
    return missing_df


# ========== Plotting Missing Values  ===========================
def na_ratio_plot(df):
    """Plots the ratio of missing values for each feature and prints the count of missing values."""
    
    sns.displot(df.isna().melt(value_name='Missing_data',var_name='Features')\
                ,y='Features',hue='Missing_data',multiple='fill',aspect=9/8)

    print(df.isna().sum()[df.isna().sum()>0])

    
    
#========== Detecting Anomalies ================================

def detect_anomalies(df, column_name):
    """
    Detects values with unusual (non-alphanumeric) characters in a column.
    Returns: list: Detected unusual character values.
    """
    # Get the unique values in the column
    unique_values = df[column_name].unique()    
    # Detect values with unusual characters (non-alphanumeric)
    unusual_characters = [val for val in unique_values if isinstance(val, str) and not val.isalnum()]
    
    # Return the list as a single string with values separated by commas
    return ', '.join(unusual_characters)


#========== Detecting Non-Numerical Characters ===========================

import re

def find_non_numeric_values(df, column_name):
    """
    Finds unique non-numeric values in a specified column of the DataFrame.
    """
    pattern = r'\D+'  # Pattern to match non-numeric characters
    # Find and flatten non-numeric values, then ensure uniqueness with set
    return set(re.findall(pattern, ' '.join(df[column_name].astype(str))))


#===============Plot all Categoricals Bar Chart ===========================
def plot_categorical_bar_graphs(df):
    """
    DataFrame'deki tüm kategorik veriler için bar grafikleri oluşturur.
    
    Args:
    - df (DataFrame): Kategorik verileri içeren DataFrame.
    """
    # Kategorik sütunları seçme
    categorical_columns = df.select_dtypes(include=['object', 'category']).columns

    # Her kategorik sütun için bar grafiği oluşturma
    for column in categorical_columns:
        plt.figure(figsize=(15, 5))
        ax = sns.countplot(
            x=column, 
            data=df, 
            order=df[column].value_counts().index, 
            palette='Set2'
        )
        ax.bar_label(ax.containers[0], rotation=45)
        plt.xlabel(column, fontsize=12)
        plt.ylabel('Count', fontsize=12)
        plt.title(f'Bar Graph of {column}', fontsize=20)
        plt.xticks(rotation=45, ha='right', fontsize=12)
        plt.show()
        
#======================================================================

def combine_dataframes(df_list):
    """
    Verilen DataFrame listesini alt alta birleştirir.
    
    :param df_list: Birleştirilecek DataFrame'lerin listesi
    :return: Birleştirilmiş DataFrame
    """
    # DataFrame'leri alt alta birleştir
    combined_df = pd.concat(df_list, ignore_index=True, sort=False)
    
    return combined_df

#====================================================================

def move_column(df, column_name, new_index):
    """
    Verilen bir sütun adını belirtilen yeni indekse taşır.
    
    :param df: DataFrame
    :param column_name: Taşınacak sütunun adı
    :param new_index: Yeni indeks numarası
    :return: Sütunları yeniden düzenlenmiş DataFrame
    """
    # Mevcut sütunları listeye al
    columns = list(df.columns)
    
    # Taşınacak sütunu kaldır
    columns.remove(column_name)
    
    # Sütunu istenilen yeni indekse ekle
    columns.insert(new_index, column_name)
    
    # DataFrame'i yeni sütun sırasına göre düzenle
    return df[columns]


# Exploratory Data Analysis (EDA)

## 2018

In [7]:
df18 = pd.read_csv("df_oksijen_18.csv")
df_oksijen_18 = df18.copy()

df_oksijen_18.head()

,DMO-ÖZEL-İHALE-DEMO,SİPARİŞ NO/İŞ EMRİ NO/TEKLİF NO,SİPARİŞ TARİHİ,SİPARİŞİ VEREN/2018,KURULUM YERİ /2018,İL,ÜRÜN,LİTRE,SERİ NO,TESLİM \nSÜRESİ,TESLİM TARİHİ,SİPARİŞİN DURUMU
0,YURTDIŞI,2018/040,2018-05-02,SIMAD UNİVERSİTESİ,UĞUR SAĞLIK HİZMETLERİ A.Ş.,SOMALİ,OXYGOLD-22,220 LT,2018/06-005,30,2018-06-10,NaN


### 'kategori' Column Ekle

In [8]:
# 'kategori' sütununu birinci indeks (ikinci sütun) olacak şekilde taşıyalım 

df_oksijen_18['kategori'] = 'OKSIJEN'
columns = ['kategori'] + list(df_oksijen_18.columns[:-1])
df_oksijen_18 = df_oksijen_18[columns]
df_oksijen_18.head(2)

,kategori,DMO-ÖZEL-İHALE-DEMO,SİPARİŞ NO/İŞ EMRİ NO/TEKLİF NO,SİPARİŞ TARİHİ,SİPARİŞİ VEREN/2018,KURULUM YERİ /2018,İL,ÜRÜN,LİTRE,SERİ NO,TESLİM \nSÜRESİ,TESLİM TARİHİ,SİPARİŞİN DURUMU
0,OKSIJEN,YURTDIŞI,2018/040,2018-05-02,SIMAD UNİVERSİTESİ,UĞUR SAĞLIK HİZMETLERİ A.Ş.,SOMALİ,OXYGOLD-22,220 LT,2018/06-005,30,2018-06-10,NaN


### Rename Columns

In [9]:
new_column_names = [
    'kategori',
    'satis_turu', 
    'siparis_no', 
    'siparis_tarihi',
    'siparisi_veren',
    'kurulum_yeri',
    'il',
    'urun_modeli',
   # 'urun_adi',
    'litre',
    'seri_no',
    'teslim_suresi',
    'teslim_tarihi',
    'siparis_durumu'
]

df_oksijen_18 = rename_columns_by_position(df_oksijen_18, new_column_names)

df_oksijen_18.head()

,kategori,satis_turu,siparis_no,siparis_tarihi,siparisi_veren,kurulum_yeri,il,urun_modeli,litre,seri_no,teslim_suresi,teslim_tarihi,siparis_durumu
0,OKSIJEN,YURTDIŞI,2018/040,2018-05-02,SIMAD UNİVERSİTESİ,UĞUR SAĞLIK HİZMETLERİ A.Ş.,SOMALİ,OXYGOLD-22,220 LT,2018/06-005,30,2018-06-10,NaN


In [ ]:
df_oksijen_18.info()

In [ ]:
# Numeric data yok
numeric_summary(df_oksijen_18)

In [ ]:
object_summary(df_oksijen_18)

In [ ]:
get_value_count(df_oksijen_18, 'teslim_suresi')

### Missings

In [ ]:
missing_values(df_oksijen_18)

### Duplicates

In [ ]:
duplicate_values(df_oksijen_18)

## 2019 

In [10]:
df19 = pd.read_csv('df_oksijen_19.csv')
df_oksijen_19 = df19.copy()

df_oksijen_19.head()

,DMO-ÖZEL-İHALE-DEMO,SİPARİŞ NO/İŞ EMRİ NO/TEKLİF NO,SİPARİŞ TARİHİ,SİPARİŞİ VEREN/2019,KURULUM YERİ /2019,İL,ÜRÜN,LİTRE,SERİ NO,TESLİM \nSÜRESİ,TESLİM TARİHİ,SİPARİŞİN DURUMU
0,YURTDIŞI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 'kategori' Column Ekle

In [11]:
# 'kategori' sütununu birinci indeks (ikinci sütun) olacak şekilde taşıyalım 

df_oksijen_19['kategori'] = 'OKSIJEN'
columns = ['kategori'] + list(df_oksijen_19.columns[:-1])
df_oksijen_19 = df_oksijen_19[columns]
df_oksijen_19.head(2)

,kategori,DMO-ÖZEL-İHALE-DEMO,SİPARİŞ NO/İŞ EMRİ NO/TEKLİF NO,SİPARİŞ TARİHİ,SİPARİŞİ VEREN/2019,KURULUM YERİ /2019,İL,ÜRÜN,LİTRE,SERİ NO,TESLİM \nSÜRESİ,TESLİM TARİHİ,SİPARİŞİN DURUMU
0,OKSIJEN,YURTDIŞI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Rename Columns

In [12]:
new_column_names = [
    'kategori',
    'satis_turu', 
    'siparis_no', 
    'siparis_tarihi',
    'siparisi_veren',
    'kurulum_yeri',
    'il',
    'urun_modeli',
   # 'urun_adi',
    'litre',
    'seri_no',
    'teslim_suresi',
    'teslim_tarihi',
    'siparis_durumu'
]

df_oksijen_19 = rename_columns_by_position(df_oksijen_19, new_column_names)

df_oksijen_19.head()

,kategori,satis_turu,siparis_no,siparis_tarihi,siparisi_veren,kurulum_yeri,il,urun_modeli,litre,seri_no,teslim_suresi,teslim_tarihi,siparis_durumu
0,OKSIJEN,YURTDIŞI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_oksijen_19.info()

In [ ]:
# Numeric data yok
numeric_summary(df_oksijen_19)

In [ ]:
object_summary(df_oksijen_19)

In [ ]:
get_value_count(df_oksijen_19, 'teslim_suresi')

### Missings

In [ ]:
missing_values(df_oksijen_19)

### Duplicates

In [ ]:
duplicate_values(df_oksijen_19)

## 2020

In [13]:
df20 = pd.read_csv('df_oksijen_20.csv')
df_oksijen_20 = df20.copy()

df_oksijen_20.head()

,DMO-ÖZEL-İHALE-DEMO,SİPARİŞ NO/İŞ EMRİ NO/TEKLİF NO,SİPARİŞ TARİHİ,SİPARİŞİ VEREN/2020,KURULUM YERİ /2018,İL,ÜRÜN,LİTRE,SERİ NO,TESLİM \nSÜRESİ,TESLİM TARİHİ,SİPARİŞİN DURUMU
0,YURTDIŞI,2020/048,2020-02-05,AL MASARA HOSPİTAL,AL MASARA HOSPİTAL,LİBYA,OXYGOLD 75,75 LT,2020/06-001,30,NaN,NaN
1,YURTDIŞI,2020/049,2020-02-05,AL MASARA HOSPİTAL,AL MASARA HOSPİTAL,LİBYA,OXYGOLD 3,NaN,2020/06-002,30,NaN,NaN
2,YURTDIŞI,2018/003,2021-01-13,"MME,LLC","MME,LLC",UKRAYNA,OXY-GOLD-11,NaN,2020/06-003,30,NaN,NaN
3,YURTDIŞI,2020/514,2021-01-13,"MME,LLC","MME,LLC",UKRAYNA,OXY-GOLD-K15,NaN,2020/06-004,30,NaN,NaN
4,YURTDIŞI,2020/515,2021-01-13,"MME,LLC","MME,LLC",UKRAYNA,OXY-GOLD-K15,NaN,2020/06-005,30,NaN,NaN


### 'kategori' Column Ekle

In [14]:
# 'kategori' sütununu birinci indeks (ikinci sütun) olacak şekilde taşıyalım 

df_oksijen_20['kategori'] = 'OKSIJEN'
columns = ['kategori'] + list(df_oksijen_20.columns[:-1])
df_oksijen_20 = df_oksijen_20[columns]
df_oksijen_20.head(2)

,kategori,DMO-ÖZEL-İHALE-DEMO,SİPARİŞ NO/İŞ EMRİ NO/TEKLİF NO,SİPARİŞ TARİHİ,SİPARİŞİ VEREN/2020,KURULUM YERİ /2018,İL,ÜRÜN,LİTRE,SERİ NO,TESLİM \nSÜRESİ,TESLİM TARİHİ,SİPARİŞİN DURUMU
0,OKSIJEN,YURTDIŞI,2020/048,2020-02-05,AL MASARA HOSPİTAL,AL MASARA HOSPİTAL,LİBYA,OXYGOLD 75,75 LT,2020/06-001,30,NaN,NaN
1,OKSIJEN,YURTDIŞI,2020/049,2020-02-05,AL MASARA HOSPİTAL,AL MASARA HOSPİTAL,LİBYA,OXYGOLD 3,NaN,2020/06-002,30,NaN,NaN


### Rename Columns

In [15]:
new_column_names = [
    'kategori',
    'satis_turu', 
    'siparis_no', 
    'siparis_tarihi',
    'siparisi_veren',
    'kurulum_yeri',
    'il',
    'urun_modeli',
   # 'urun_adi',
    'litre',
    'seri_no',
    'teslim_suresi',
    'teslim_tarihi',
    'siparis_durumu'
]

df_oksijen_20 = rename_columns_by_position(df_oksijen_20, new_column_names)

df_oksijen_20.head()

,kategori,satis_turu,siparis_no,siparis_tarihi,siparisi_veren,kurulum_yeri,il,urun_modeli,litre,seri_no,teslim_suresi,teslim_tarihi,siparis_durumu
0,OKSIJEN,YURTDIŞI,2020/048,2020-02-05,AL MASARA HOSPİTAL,AL MASARA HOSPİTAL,LİBYA,OXYGOLD 75,75 LT,2020/06-001,30,NaN,NaN
1,OKSIJEN,YURTDIŞI,2020/049,2020-02-05,AL MASARA HOSPİTAL,AL MASARA HOSPİTAL,LİBYA,OXYGOLD 3,NaN,2020/06-002,30,NaN,NaN
2,OKSIJEN,YURTDIŞI,2018/003,2021-01-13,"MME,LLC","MME,LLC",UKRAYNA,OXY-GOLD-11,NaN,2020/06-003,30,NaN,NaN
3,OKSIJEN,YURTDIŞI,2020/514,2021-01-13,"MME,LLC","MME,LLC",UKRAYNA,OXY-GOLD-K15,NaN,2020/06-004,30,NaN,NaN
4,OKSIJEN,YURTDIŞI,2020/515,2021-01-13,"MME,LLC","MME,LLC",UKRAYNA,OXY-GOLD-K15,NaN,2020/06-005,30,NaN,NaN


In [ ]:
df_oksijen_20.info()

In [ ]:
# Numeric data yok
numeric_summary(df_oksijen_20)

In [ ]:
object_summary(df_oksijen_20)

In [ ]:
get_value_count(df_oksijen_20, 'teslim_suresi')

### Missings

In [ ]:
missing_values(df_oksijen_20)

### Duplicates

In [ ]:
duplicate_values(df_oksijen_20)

## 2021

In [16]:
df21 = pd.read_csv('df_oksijen_21.csv')
df_oksijen_21 = df21.copy()

df_oksijen_21.head()

,DMO-ÖZEL-İHALE-DEMO,SİPARİŞ NO/İŞ EMRİ NO/TEKLİF NO,SİPARİŞ TARİHİ,SİPARİŞİ VEREN/2020,KURULUM YERİ /2018,İL,ÜRÜN,LİTRE,SERİ NO,TESLİM TARİHİ,TESLİM SURESI,SİPARİŞİN DURUMU
0,YURTDIŞI,2021/181,2021-05-26,NHAVA SHEVA,NHAVA SHEVA,HİNDİSTAN,OXY-GOLD 37,NaN,2021/06-001,2021-06-06 00:00:00,NaN,NaN
1,YURTDIŞI,2021/035,2021-02-19,"MME,LLC(MED LOGİSTİCS)","MME,LLC(MED LOGİSTİCS)",UKRAYNA,OXY-GOLD 37,NaN,2021/06-002,2021-03-10 00:00:00,NaN,NaN
2,YURTDIŞI,2021/036,2021-02-19,"MME,LLC(MED LOGİSTİCS)","MME,LLC(MED LOGİSTİCS)",UKRAYNA,OXY-GOLD 15,NaN,2021/06-003,2021-03-10 00:00:00,NaN,NaN
3,YURTDIŞI,2021/037,2021-02-19,"MME,LLC(MED LOGİSTİCS)","MME,LLC(MED LOGİSTİCS)",UKRAYNA,OXY-GOLD 18,NaN,2021/06-004,2021-03-10 00:00:00,NaN,NaN
4,YURTDIŞI,2021/038,2021-02-19,"MME,LLC(MED LOGİSTİCS)","MME,LLC(MED LOGİSTİCS)",UKRAYNA,OXY-GOLD 30,NaN,2021/06-005,2021-03-10 00:00:00,NaN,NaN


### 'kategori' Column Ekle

In [17]:
# 'kategori' sütununu birinci indeks  olacak şekilde taşıyalım 

df_oksijen_21['kategori'] = 'OKSIJEN'
columns = ['kategori'] + list(df_oksijen_21.columns[:-1])
df_oksijen_21 = df_oksijen_21[columns]
df_oksijen_21.head(2)

,kategori,DMO-ÖZEL-İHALE-DEMO,SİPARİŞ NO/İŞ EMRİ NO/TEKLİF NO,SİPARİŞ TARİHİ,SİPARİŞİ VEREN/2020,KURULUM YERİ /2018,İL,ÜRÜN,LİTRE,SERİ NO,TESLİM TARİHİ,TESLİM SURESI,SİPARİŞİN DURUMU
0,OKSIJEN,YURTDIŞI,2021/181,2021-05-26,NHAVA SHEVA,NHAVA SHEVA,HİNDİSTAN,OXY-GOLD 37,NaN,2021/06-001,2021-06-06 00:00:00,NaN,NaN
1,OKSIJEN,YURTDIŞI,2021/035,2021-02-19,"MME,LLC(MED LOGİSTİCS)","MME,LLC(MED LOGİSTİCS)",UKRAYNA,OXY-GOLD 37,NaN,2021/06-002,2021-03-10 00:00:00,NaN,NaN


### Rename Columns

In [18]:
new_column_names = [
    'kategori',
    'satis_turu', 
    'siparis_no', 
    'siparis_tarihi',
    'siparisi_veren',
    'kurulum_yeri',
    'il',
    'urun_modeli',
   # 'urun_adi',
    'litre',
    'seri_no',
    'teslim_suresi',
    'teslim_tarihi',
    'siparis_durumu'
]

df_oksijen_21 = rename_columns_by_position(df_oksijen_21, new_column_names)

df_oksijen_21.head()

,kategori,satis_turu,siparis_no,siparis_tarihi,siparisi_veren,kurulum_yeri,il,urun_modeli,litre,seri_no,teslim_suresi,teslim_tarihi,siparis_durumu
0,OKSIJEN,YURTDIŞI,2021/181,2021-05-26,NHAVA SHEVA,NHAVA SHEVA,HİNDİSTAN,OXY-GOLD 37,NaN,2021/06-001,2021-06-06 00:00:00,NaN,NaN
1,OKSIJEN,YURTDIŞI,2021/035,2021-02-19,"MME,LLC(MED LOGİSTİCS)","MME,LLC(MED LOGİSTİCS)",UKRAYNA,OXY-GOLD 37,NaN,2021/06-002,2021-03-10 00:00:00,NaN,NaN
2,OKSIJEN,YURTDIŞI,2021/036,2021-02-19,"MME,LLC(MED LOGİSTİCS)","MME,LLC(MED LOGİSTİCS)",UKRAYNA,OXY-GOLD 15,NaN,2021/06-003,2021-03-10 00:00:00,NaN,NaN
3,OKSIJEN,YURTDIŞI,2021/037,2021-02-19,"MME,LLC(MED LOGİSTİCS)","MME,LLC(MED LOGİSTİCS)",UKRAYNA,OXY-GOLD 18,NaN,2021/06-004,2021-03-10 00:00:00,NaN,NaN
4,OKSIJEN,YURTDIŞI,2021/038,2021-02-19,"MME,LLC(MED LOGİSTİCS)","MME,LLC(MED LOGİSTİCS)",UKRAYNA,OXY-GOLD 30,NaN,2021/06-005,2021-03-10 00:00:00,NaN,NaN


In [ ]:
df_oksijen_21.info()

In [ ]:
# Numeric data yok
numeric_summary(df_oksijen_21)

In [ ]:
object_summary(df_oksijen_21)

In [ ]:
get_value_count(df_oksijen_21, 'teslim_suresi')

### Missings

In [ ]:
missing_values(df_oksijen_21)

### Duplicates

In [ ]:
duplicate_values(df_oksijen_21)

## 2022 

In [19]:
df22 = pd.read_csv('df_oksijen_22.csv')
df_oksijen_22 = df22.copy()

df_oksijen_22.head()

,DMO-ÖZEL-İHALE-DEMO,SİPARİŞ NO/İŞ EMRİ NO/TEKLİF NO,SİPARİŞ TARİHİ,SİPARİŞİ VEREN/2022,KURULUM YERİ /2022,İL,ÜRÜN MODEL,ÜRÜN,LİTRE,SERİ NO,TESLİM \nSÜRESİ,TESLİM TARİHİ,İRSALİYE TARİHİ
0,YURTDIŞI,2022/153,2022-02-02,DUTCH,LİBERYA,LİBERYA,GOLDBERG 45 V,45 OKSİJEN ÜRETİM SİSTEMİ,NaN,2022/06-003,206,2022-08-27,NaN
1,ÖZEL,2022/160,2022-02-02,BİOKAN SAĞLIK ÜRÜNLERİ SAN. VE TİC. A.Ş.,BİOKAN SAĞLIK ÜRÜNLERİ SAN. VE TİC.A.Ş.,KAYSERİ,STR 22 V,22 OKSİJEN ÜRETİM SİSTEMİ,NaN,2022/06-002,176,2022-07-28,NaN
2,YURTDIŞI,2022/364,2022-02-02,IRAK,IRAK,IRAK,GOLDBERG 22 V,22 OKSİJEN ÜRETİM SİSTEMİ,NaN,2022/06-006,267,2022-10-27,NaN


### 'kategori' Column Ekle

In [20]:
# 'kategori' sütununu birinci indeks (ikinci sütun) olacak şekilde taşıyalım 

df_oksijen_22['kategori'] = 'OKSIJEN'
columns = ['kategori'] + list(df_oksijen_22.columns[:-1])
df_oksijen_22 = df_oksijen_22[columns]
df_oksijen_22.head(2)

,kategori,DMO-ÖZEL-İHALE-DEMO,SİPARİŞ NO/İŞ EMRİ NO/TEKLİF NO,SİPARİŞ TARİHİ,SİPARİŞİ VEREN/2022,KURULUM YERİ /2022,İL,ÜRÜN MODEL,ÜRÜN,LİTRE,SERİ NO,TESLİM \nSÜRESİ,TESLİM TARİHİ,İRSALİYE TARİHİ
0,OKSIJEN,YURTDIŞI,2022/153,2022-02-02,DUTCH,LİBERYA,LİBERYA,GOLDBERG 45 V,45 OKSİJEN ÜRETİM SİSTEMİ,NaN,2022/06-003,206,2022-08-27,NaN
1,OKSIJEN,ÖZEL,2022/160,2022-02-02,BİOKAN SAĞLIK ÜRÜNLERİ SAN. VE TİC. A.Ş.,BİOKAN SAĞLIK ÜRÜNLERİ SAN. VE TİC.A.Ş.,KAYSERİ,STR 22 V,22 OKSİJEN ÜRETİM SİSTEMİ,NaN,2022/06-002,176,2022-07-28,NaN


### Rename Columns

In [22]:
new_column_names = [
    'kategori',
    'satis_turu', 
    'siparis_no', 
    'siparis_tarihi',
    'siparisi_veren',
    'kurulum_yeri',
    'il',
    'urun_modeli',
    'urun_adi',
    'litre',
    'seri_no',
    'teslim_suresi',
    'teslim_tarihi',
    'siparis_durumu'
]

df_oksijen_22 = rename_columns_by_position(df_oksijen_22, new_column_names)

df_oksijen_22.head()

,kategori,satis_turu,siparis_no,siparis_tarihi,siparisi_veren,kurulum_yeri,il,urun_modeli,urun_adi,litre,seri_no,teslim_suresi,teslim_tarihi,siparis_durumu
0,OKSIJEN,YURTDIŞI,2022/153,2022-02-02,DUTCH,LİBERYA,LİBERYA,GOLDBERG 45 V,45 OKSİJEN ÜRETİM SİSTEMİ,NaN,2022/06-003,206,2022-08-27,NaN
1,OKSIJEN,ÖZEL,2022/160,2022-02-02,BİOKAN SAĞLIK ÜRÜNLERİ SAN. VE TİC. A.Ş.,BİOKAN SAĞLIK ÜRÜNLERİ SAN. VE TİC.A.Ş.,KAYSERİ,STR 22 V,22 OKSİJEN ÜRETİM SİSTEMİ,NaN,2022/06-002,176,2022-07-28,NaN
2,OKSIJEN,YURTDIŞI,2022/364,2022-02-02,IRAK,IRAK,IRAK,GOLDBERG 22 V,22 OKSİJEN ÜRETİM SİSTEMİ,NaN,2022/06-006,267,2022-10-27,NaN


In [ ]:
df_oksijen_22.info()

In [ ]:
# Numeric data yok
numeric_summary(df_oksijen_22)

In [ ]:
object_summary(df_oksijen_22)

In [ ]:
get_value_count(df_oksijen_22, 'teslim_suresi')

### Missings

In [ ]:
missing_values(df_oksijen_22)

### Duplicates

In [ ]:
duplicate_values(df_oksijen_22)

## 2023 

## 2023_endüstriyel

In [23]:
df23 = pd.read_csv('df_oksijen_23.csv')
df_oksijen_23 = df23.copy()

df_oksijen_23.head()

,DMO-ÖZEL-İHALE-DEMO,SİPARİŞ NO/İŞ EMRİ NO/TEKLİF NO,SİPARİŞ TARİHİ,SİPARİŞİ VEREN/2022,KURULUM YERİ /2022,İL,ÜRÜN MODEL,ÜRÜN,LİTRE,SERİ NO,TESLİM \nSÜRESİ,TESLİM TARİHİ,İRSALİYE TARİHİ
0,YURTDIŞI,2022/153,2022-02-02,DUTCH,LİBERYA,LİBERYA,GOLDBERG 45 V,45 OKSİJEN ÜRETİM SİSTEMİ,NaN,2022/06-003,206,2022-08-27,NaN
1,ÖZEL,2022/160,2022-02-02,BİOKAN SAĞLIK ÜRÜNLERİ SAN. VE TİC. A.Ş.,BİOKAN SAĞLIK ÜRÜNLERİ SAN. VE TİC.A.Ş.,KAYSERİ,STR 22 V,22 OKSİJEN ÜRETİM SİSTEMİ,NaN,2022/06-002,176,2022-07-28,NaN
2,YURTDIŞI,2022/364,2022-02-02,IRAK,IRAK,IRAK,GOLDBERG 22 V,22 OKSİJEN ÜRETİM SİSTEMİ,NaN,2022/06-006,267,2022-10-27,NaN


### 'kategori' Column Ekle

In [24]:
# 'kategori' sütununu birinci indeks (ikinci sütun) olacak şekilde taşıyalım 

df_oksijen_23['kategori'] = 'OKSIJEN'
columns = ['kategori'] + list(df_oksijen_23.columns[:-1])
df_oksijen_23 = df_oksijen_23[columns]
df_oksijen_23.head(2)

,kategori,DMO-ÖZEL-İHALE-DEMO,SİPARİŞ NO/İŞ EMRİ NO/TEKLİF NO,SİPARİŞ TARİHİ,SİPARİŞİ VEREN/2022,KURULUM YERİ /2022,İL,ÜRÜN MODEL,ÜRÜN,LİTRE,SERİ NO,TESLİM \nSÜRESİ,TESLİM TARİHİ,İRSALİYE TARİHİ
0,OKSIJEN,YURTDIŞI,2022/153,2022-02-02,DUTCH,LİBERYA,LİBERYA,GOLDBERG 45 V,45 OKSİJEN ÜRETİM SİSTEMİ,NaN,2022/06-003,206,2022-08-27,NaN
1,OKSIJEN,ÖZEL,2022/160,2022-02-02,BİOKAN SAĞLIK ÜRÜNLERİ SAN. VE TİC. A.Ş.,BİOKAN SAĞLIK ÜRÜNLERİ SAN. VE TİC.A.Ş.,KAYSERİ,STR 22 V,22 OKSİJEN ÜRETİM SİSTEMİ,NaN,2022/06-002,176,2022-07-28,NaN


### Rename Columns

In [25]:
new_column_names = [
    'kategori',
    'satis_turu', 
    'siparis_no', 
    'siparis_tarihi',
    'siparisi_veren',
    'kurulum_yeri',
    'il',
    'urun_modeli',
    'urun_adi',
    'litre',
    'seri_no',
    'teslim_suresi',
    'teslim_tarihi',
    'siparis_durumu'
]

df_oksijen_23 = rename_columns_by_position(df_oksijen_23, new_column_names)

df_oksijen_23.head()

,kategori,satis_turu,siparis_no,siparis_tarihi,siparisi_veren,kurulum_yeri,il,urun_modeli,urun_adi,litre,seri_no,teslim_suresi,teslim_tarihi,siparis_durumu
0,OKSIJEN,YURTDIŞI,2022/153,2022-02-02,DUTCH,LİBERYA,LİBERYA,GOLDBERG 45 V,45 OKSİJEN ÜRETİM SİSTEMİ,NaN,2022/06-003,206,2022-08-27,NaN
1,OKSIJEN,ÖZEL,2022/160,2022-02-02,BİOKAN SAĞLIK ÜRÜNLERİ SAN. VE TİC. A.Ş.,BİOKAN SAĞLIK ÜRÜNLERİ SAN. VE TİC.A.Ş.,KAYSERİ,STR 22 V,22 OKSİJEN ÜRETİM SİSTEMİ,NaN,2022/06-002,176,2022-07-28,NaN
2,OKSIJEN,YURTDIŞI,2022/364,2022-02-02,IRAK,IRAK,IRAK,GOLDBERG 22 V,22 OKSİJEN ÜRETİM SİSTEMİ,NaN,2022/06-006,267,2022-10-27,NaN


In [ ]:
df_oksijen_23.info()

In [ ]:
# Numeric data yok
numeric_summary(df_oksijen_23)

In [ ]:
object_summary(df_oksijen_23)

In [ ]:
get_value_count(df_oksijen_23, 'teslim_suresi')

### Missings

In [ ]:
missing_values(df_oksijen_23)

### Duplicates

In [ ]:
duplicate_values(df_oksijen_23)

## 2024 

In [26]:
df24 = pd.read_csv('df_oksijen_24.csv')
df_oksijen_24 = df24.copy()

df_oksijen_24.head()

,DMO-ÖZEL-İHALE-DEMO,SİPARİŞ NO/İŞ EMRİ NO/TEKLİF NO,SİPARİŞ TARİHİ,SİPARİŞİ VEREN/2024,KURULUM YERİ /2024,İL,ÜRÜN MODEL,ÜRÜN,LİTRE,SERİ NO,TESLİM \nSÜRESİ,TESLİM TARİHİ,İRSALİYE TARİHİ,ÜTS NO
0,YURTDIŞI,2022/156,2024-02-27,CEZAYİR,CEZAYİR,CEZAYİR,OXY-GOLD 37,OXY-GOLD 37 OKSİJEN ÜRETİM VE DEPOLAMA SİSTEMİ,NaN,2024/06-001,60 gün,2024-03-11,NaN,8.697613e+12
1,YURTDIŞI,2022/161,2022-03-03,STOK,STOK,STOK,OXY-GOLD 30,OXY-GOLD 30 OKSİJEN ÜRETİM VE DEPOLAMA SİSTEMİ,NaN,2024/06-003,60 gün,2024-03-12,NaN,8.697613e+12
2,YURTDIŞI,2022/163,2024-06-11,CEZAYİR,CEZAYİR,CEZAYİR,OXY-GOLD 30,OXY-GOLD 30 OKSİJEN ÜRETİM VE DEPOLAMA SİSTEMİ,NaN,2024/06-005,60 gün,2024-06-13,NaN,8.697613e+12
3,YURTDIŞI,2024/283,2024-09-25,CEZAYİR,CEZAYİR,CEZAYİR,OXY-GOLD 37,OXY-GOLD 37 OKSİJEN ÜRETİM VE DEPOLAMA SİSTEMİ,NaN,2024/06-006,60 gün,2024-10-13,NaN,NaN


### 'kategori' Column Ekle

In [27]:
# 'kategori' sütununu birinci indeks (ikinci sütun) olacak şekilde taşıyalım 

df_oksijen_24['kategori'] = 'OKSIJEN'
columns = ['kategori'] + list(df_oksijen_24.columns[:-1])
df_oksijen_24 = df_oksijen_24[columns]
df_oksijen_24.head(2)

,kategori,DMO-ÖZEL-İHALE-DEMO,SİPARİŞ NO/İŞ EMRİ NO/TEKLİF NO,SİPARİŞ TARİHİ,SİPARİŞİ VEREN/2024,KURULUM YERİ /2024,İL,ÜRÜN MODEL,ÜRÜN,LİTRE,SERİ NO,TESLİM \nSÜRESİ,TESLİM TARİHİ,İRSALİYE TARİHİ,ÜTS NO
0,OKSIJEN,YURTDIŞI,2022/156,2024-02-27,CEZAYİR,CEZAYİR,CEZAYİR,OXY-GOLD 37,OXY-GOLD 37 OKSİJEN ÜRETİM VE DEPOLAMA SİSTEMİ,NaN,2024/06-001,60 gün,2024-03-11,NaN,8.697613e+12
1,OKSIJEN,YURTDIŞI,2022/161,2022-03-03,STOK,STOK,STOK,OXY-GOLD 30,OXY-GOLD 30 OKSİJEN ÜRETİM VE DEPOLAMA SİSTEMİ,NaN,2024/06-003,60 gün,2024-03-12,NaN,8.697613e+12


In [ ]:
# 'TESLİM TARİHİ' sütununu DataFrame'den silme (ham verimizde yok)
#df_oksijen_24 = df_oksijen_24.drop('TESLİM TARİHİ', axis=1)

### Rename Columns

In [30]:
new_column_names = [
    'kategori',
    'satis_turu', 
    'siparis_no', 
    'siparis_tarihi',
    'siparisi_veren',
    'kurulum_yeri',
    'il',
    'urun_modeli',
    'urun_adi',
    'litre',
    'seri_no',
    'teslim_suresi',
    'teslim_tarihi',
    #'siparis_durumu',
    'irsaliye_tarihi',
    'uts_no'
]

df_oksijen_24 = rename_columns_by_position(df_oksijen_24, new_column_names)

df_oksijen_24.head()

,kategori,satis_turu,siparis_no,siparis_tarihi,siparisi_veren,kurulum_yeri,il,urun_modeli,urun_adi,litre,seri_no,teslim_suresi,teslim_tarihi,irsaliye_tarihi,uts_no
0,OKSIJEN,YURTDIŞI,2022/156,2024-02-27,CEZAYİR,CEZAYİR,CEZAYİR,OXY-GOLD 37,OXY-GOLD 37 OKSİJEN ÜRETİM VE DEPOLAMA SİSTEMİ,NaN,2024/06-001,60 gün,2024-03-11,NaN,8.697613e+12
1,OKSIJEN,YURTDIŞI,2022/161,2022-03-03,STOK,STOK,STOK,OXY-GOLD 30,OXY-GOLD 30 OKSİJEN ÜRETİM VE DEPOLAMA SİSTEMİ,NaN,2024/06-003,60 gün,2024-03-12,NaN,8.697613e+12
2,OKSIJEN,YURTDIŞI,2022/163,2024-06-11,CEZAYİR,CEZAYİR,CEZAYİR,OXY-GOLD 30,OXY-GOLD 30 OKSİJEN ÜRETİM VE DEPOLAMA SİSTEMİ,NaN,2024/06-005,60 gün,2024-06-13,NaN,8.697613e+12
3,OKSIJEN,YURTDIŞI,2024/283,2024-09-25,CEZAYİR,CEZAYİR,CEZAYİR,OXY-GOLD 37,OXY-GOLD 37 OKSİJEN ÜRETİM VE DEPOLAMA SİSTEMİ,NaN,2024/06-006,60 gün,2024-10-13,NaN,NaN


In [ ]:
df_oksijen_24.info()

In [ ]:
numeric_summary(df_oksijen_24)

In [ ]:
object_summary(df_oksijen_24)

In [ ]:
get_value_count(df_oksijen_24, 'teslim_suresi')

### Missings

In [ ]:
missing_values(df_oksijen_24)

### Duplicates

In [ ]:
duplicate_values(df_oksijen_24)

# **Tüm dosyaları kayıt etme** 

In [31]:
import os

# DataFrame isimlerini ve dosya isimlerini bir araya getirme
dfs = [
    (df_oksijen_18, "df_oksijen_18.csv"),
    (df_oksijen_19, "df_oksijen_19.csv"),
    (df_oksijen_20, "df_oksijen_20.csv"),
    (df_oksijen_21, "df_oksijen_21.csv"),
    (df_oksijen_22, "df_oksijen_22.csv"),
    (df_oksijen_23, "df_oksijen_23.csv"),
    (df_oksijen_24, "df_oksijen_23.csv")
]

# Belirtilen dosya yolunu tanımlama
save_path = r'C:\Users\duygu\Desktop\00-GitHub-Repo\00-1INTERNSHIP\INT1_PROJECT\01_EDA\DATA INCELEME\03_OKSİJEN SİSTEMİ\clean_csv'

# Dosya yolunu oluşturma (eğer yoksa)
os.makedirs(save_path, exist_ok=True)

# DataFrame'leri CSV olarak kaydetme
file_paths = []  # Kaydedilen dosya yollarını tutmak için
for df, filename in dfs:
    file_path = os.path.join(save_path, filename)
    df.to_csv(file_path, index=False, encoding='utf-8-sig')
    file_paths.append(file_path)

file_paths  # Kaydedilen dosya yollarını göstereceğiz

['C:\\Users\\duygu\\Desktop\\00-GitHub-Repo\\00-1INTERNSHIP\\INT1_PROJECT\\01_EDA\\DATA INCELEME\\03_OKSİJEN SİSTEMİ\\clean_csv\\df_oksijen_18.csv',
 'C:\\Users\\duygu\\Desktop\\00-GitHub-Repo\\00-1INTERNSHIP\\INT1_PROJECT\\01_EDA\\DATA INCELEME\\03_OKSİJEN SİSTEMİ\\clean_csv\\df_oksijen_19.csv',
 'C:\\Users\\duygu\\Desktop\\00-GitHub-Repo\\00-1INTERNSHIP\\INT1_PROJECT\\01_EDA\\DATA INCELEME\\03_OKSİJEN SİSTEMİ\\clean_csv\\df_oksijen_20.csv',
 'C:\\Users\\duygu\\Desktop\\00-GitHub-Repo\\00-1INTERNSHIP\\INT1_PROJECT\\01_EDA\\DATA INCELEME\\03_OKSİJEN SİSTEMİ\\clean_csv\\df_oksijen_21.csv',
 'C:\\Users\\duygu\\Desktop\\00-GitHub-Repo\\00-1INTERNSHIP\\INT1_PROJECT\\01_EDA\\DATA INCELEME\\03_OKSİJEN SİSTEMİ\\clean_csv\\df_oksijen_22.csv',
 'C:\\Users\\duygu\\Desktop\\00-GitHub-Repo\\00-1INTERNSHIP\\INT1_PROJECT\\01_EDA\\DATA INCELEME\\03_OKSİJEN SİSTEMİ\\clean_csv\\df_oksijen_23.csv',
 'C:\\Users\\duygu\\Desktop\\00-GitHub-Repo\\00-1INTERNSHIP\\INT1_PROJECT\\01_EDA\\DATA INCELEME\\03_OKSİJ

In [34]:
import os

# DataFrame isimlerini ve dosya isimlerini bir araya getirme
dfs = [
    (df_oksijen_18, "df_oksijen_18.xlsx"),
    (df_oksijen_19, "df_oksijen_19.xlsx"),
    (df_oksijen_20, "df_oksijen_20.xlsx"),
    (df_oksijen_21, "df_oksijen_21.xlsx"),
    (df_oksijen_22, "df_oksijen_22.xlsx"),
    (df_oksijen_23, "df_oksijen_23.xlsx"),
    (df_oksijen_24, "df_oksijen_24.xlsx")
]

# Belirtilen dosya yolunu tanımlama
save_path = r'C:\Users\duygu\Desktop\00-GitHub-Repo\00-1INTERNSHIP\INT1_PROJECT\01_EDA\DATA INCELEME\03_OKSİJEN SİSTEMİ\clean_csv'

# Dosya yolunu oluşturma (eğer yoksa)
os.makedirs(save_path, exist_ok=True)

# DataFrame'leri Excel olarak kaydetme
file_paths = []  # Kaydedilen dosya yollarını tutmak için
for df, filename in dfs:
    file_path = os.path.join(save_path, filename)
    df.to_excel(file_path, index=False)
    file_paths.append(file_path)

file_paths  # Kaydedilen dosya yollarını göstereceğiz


['C:\\Users\\duygu\\Desktop\\00-GitHub-Repo\\00-1INTERNSHIP\\INT1_PROJECT\\01_EDA\\DATA INCELEME\\03_OKSİJEN SİSTEMİ\\clean_csv\\df_oksijen_18.xlsx',
 'C:\\Users\\duygu\\Desktop\\00-GitHub-Repo\\00-1INTERNSHIP\\INT1_PROJECT\\01_EDA\\DATA INCELEME\\03_OKSİJEN SİSTEMİ\\clean_csv\\df_oksijen_19.xlsx',
 'C:\\Users\\duygu\\Desktop\\00-GitHub-Repo\\00-1INTERNSHIP\\INT1_PROJECT\\01_EDA\\DATA INCELEME\\03_OKSİJEN SİSTEMİ\\clean_csv\\df_oksijen_20.xlsx',
 'C:\\Users\\duygu\\Desktop\\00-GitHub-Repo\\00-1INTERNSHIP\\INT1_PROJECT\\01_EDA\\DATA INCELEME\\03_OKSİJEN SİSTEMİ\\clean_csv\\df_oksijen_21.xlsx',
 'C:\\Users\\duygu\\Desktop\\00-GitHub-Repo\\00-1INTERNSHIP\\INT1_PROJECT\\01_EDA\\DATA INCELEME\\03_OKSİJEN SİSTEMİ\\clean_csv\\df_oksijen_22.xlsx',
 'C:\\Users\\duygu\\Desktop\\00-GitHub-Repo\\00-1INTERNSHIP\\INT1_PROJECT\\01_EDA\\DATA INCELEME\\03_OKSİJEN SİSTEMİ\\clean_csv\\df_oksijen_23.xlsx',
 'C:\\Users\\duygu\\Desktop\\00-GitHub-Repo\\00-1INTERNSHIP\\INT1_PROJECT\\01_EDA\\DATA INCELEME\\03

In [36]:
# Ornek df goster

import pandas as pd

# Dosya yolunu belirtelim
file_path = r'C:\Users\duygu\Desktop\00-GitHub-Repo\00-1INTERNSHIP\INT1_PROJECT\01_EDA\DATA INCELEME\03_OKSİJEN SİSTEMİ\clean_csv\df_oksijen_23.csv'

# CSV dosyasını okuyalım
try:
    df_oksijen_23_read = pd.read_csv(file_path)
    # İlk birkaç satırı göstereceğiz
    df_oksijen_23_read.head()
except FileNotFoundError:
    "Dosya yolunda bir hata olabilir veya dosya mevcut olmayabilir."
    
df_oksijen_23_read.head()


,kategori,satis_turu,siparis_no,siparis_tarihi,siparisi_veren,kurulum_yeri,il,urun_modeli,urun_adi,litre,seri_no,teslim_suresi,teslim_tarihi,irsaliye_tarihi,uts_no
0,OKSIJEN,YURTDIŞI,2022/156,2024-02-27,CEZAYİR,CEZAYİR,CEZAYİR,OXY-GOLD 37,OXY-GOLD 37 OKSİJEN ÜRETİM VE DEPOLAMA SİSTEMİ,NaN,2024/06-001,60 gün,2024-03-11,NaN,8.697613e+12
1,OKSIJEN,YURTDIŞI,2022/161,2022-03-03,STOK,STOK,STOK,OXY-GOLD 30,OXY-GOLD 30 OKSİJEN ÜRETİM VE DEPOLAMA SİSTEMİ,NaN,2024/06-003,60 gün,2024-03-12,NaN,8.697613e+12
2,OKSIJEN,YURTDIŞI,2022/163,2024-06-11,CEZAYİR,CEZAYİR,CEZAYİR,OXY-GOLD 30,OXY-GOLD 30 OKSİJEN ÜRETİM VE DEPOLAMA SİSTEMİ,NaN,2024/06-005,60 gün,2024-06-13,NaN,8.697613e+12
3,OKSIJEN,YURTDIŞI,2024/283,2024-09-25,CEZAYİR,CEZAYİR,CEZAYİR,OXY-GOLD 37,OXY-GOLD 37 OKSİJEN ÜRETİM VE DEPOLAMA SİSTEMİ,NaN,2024/06-006,60 gün,2024-10-13,NaN,NaN


# **Tüm Yılları CONCAT Etme** 

In [41]:
# Tüm DataFrame'leri alt alta birleştiriyoruz
dfs_list = [df_oksijen_18,df_oksijen_19, df_oksijen_20, df_oksijen_21, df_oksijen_22, df_oksijen_23, df_oksijen_24,]

combined_df = combine_dataframes(dfs_list)

# Sonuç DataFrame'i görüntülemek için
combined_df

,kategori,satis_turu,siparis_no,siparis_tarihi,siparisi_veren,kurulum_yeri,il,urun_modeli,litre,seri_no,teslim_suresi,teslim_tarihi,siparis_durumu,urun_adi,irsaliye_tarihi,uts_no
0,OKSIJEN,YURTDIŞI,2018/040,2018-05-02,SIMAD UNİVERSİTESİ,UĞUR SAĞLIK HİZMETLERİ A.Ş.,SOMALİ,OXYGOLD-22,220 LT,2018/06-005,30,2018-06-10,NaN,NaN,NaN,NaN
1,OKSIJEN,YURTDIŞI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,OKSIJEN,YURTDIŞI,2020/048,2020-02-05,AL MASARA HOSPİTAL,AL MASARA HOSPİTAL,LİBYA,OXYGOLD 75,75 LT,2020/06-001,30,NaN,NaN,NaN,NaN,NaN
3,OKSIJEN,YURTDIŞI,2020/049,2020-02-05,AL MASARA HOSPİTAL,AL MASARA HOSPİTAL,LİBYA,OXYGOLD 3,NaN,2020/06-002,30,NaN,NaN,NaN,NaN,NaN
4,OKSIJEN,YURTDIŞI,2018/003,2021-01-13,"MME,LLC","MME,LLC",UKRAYNA,OXY-GOLD-11,NaN,2020/06-003,30,NaN,NaN,NaN,NaN,NaN
5,OKSIJEN,YURTDIŞI,2020/514,2021-01-13,"MME,LLC","MME,LLC",UKRAYNA,OXY-GOLD-K15,NaN,2020/06-004,30,NaN,NaN,NaN,NaN,NaN
6,OKSIJEN,YURTDIŞI,2020/515,2021-01-13,"MME,LLC","MME,LLC",UKRAYNA,OXY-GOLD-K15,NaN,2020/06-005,30,NaN,NaN,NaN,NaN,NaN
7,OKSIJEN,YURTDIŞI,2021/181,2021-05-26,NHAVA SHEVA,NHAVA SHEVA,HİNDİSTAN,OXY-GOLD 37,NaN,2021/06-001,2021-06-06 00:00:00,NaN,NaN,NaN,NaN,NaN
8,OKSIJEN,YURTDIŞI,2021/035,2021-02-19,"MME,LLC(MED LOGİSTİCS)","MME,LLC(MED LOGİSTİCS)",UKRAYNA,OXY-GOLD 37,NaN,2021/06-002,2021-03-10 00:00:00,NaN,NaN,NaN,NaN,NaN
9,OKSIJEN,YURTDIŞI,2021/036,2021-02-19,"MME,LLC(MED LOGİSTİCS)","MME,LLC(MED LOGİSTİCS)",UKRAYNA,OXY-GOLD 15,NaN,2021/06-003,2021-03-10 00:00:00,NaN,NaN,NaN,NaN,NaN


In [38]:
combined_df.columns

Index(['kategori', 'satis_turu', 'siparis_no', 'siparis_tarihi', 'siparisi_veren', 'kurulum_yeri', 'il', 'urun_modeli', 'litre', 'seri_no', 'teslim_suresi', 'teslim_tarihi', 'siparis_durumu', 'urun_adi', 'irsaliye_tarihi', 'uts_no'], dtype='object')

In [43]:
# Move Colunms index Fonksiyonu cagir

# combined_df üzerinde 'urun_adi' 'urun_modeli' nden sonra getirmek icin
combined_df = move_column(combined_df, 'urun_adi', 8)

combined_df.head()


,kategori,satis_turu,siparis_no,siparis_tarihi,siparisi_veren,kurulum_yeri,il,urun_modeli,urun_adi,litre,seri_no,teslim_suresi,teslim_tarihi,siparis_durumu,irsaliye_tarihi,uts_no
0,OKSIJEN,YURTDIŞI,2018/040,2018-05-02,SIMAD UNİVERSİTESİ,UĞUR SAĞLIK HİZMETLERİ A.Ş.,SOMALİ,OXYGOLD-22,NaN,220 LT,2018/06-005,30,2018-06-10,NaN,NaN,NaN
1,OKSIJEN,YURTDIŞI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,OKSIJEN,YURTDIŞI,2020/048,2020-02-05,AL MASARA HOSPİTAL,AL MASARA HOSPİTAL,LİBYA,OXYGOLD 75,NaN,75 LT,2020/06-001,30,NaN,NaN,NaN,NaN
3,OKSIJEN,YURTDIŞI,2020/049,2020-02-05,AL MASARA HOSPİTAL,AL MASARA HOSPİTAL,LİBYA,OXYGOLD 3,NaN,NaN,2020/06-002,30,NaN,NaN,NaN,NaN
4,OKSIJEN,YURTDIŞI,2018/003,2021-01-13,"MME,LLC","MME,LLC",UKRAYNA,OXY-GOLD-11,NaN,NaN,2020/06-003,30,NaN,NaN,NaN,NaN


In [44]:
combined_df.to_csv('combined_oksijen.csv', index=False, encoding='utf-8-sig')